In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
data = pd.read_csv('rent_properties_london.csv')
data.head()

,PropertyType,Address,Description,Bedrooms,Bathrooms,Price GBP,Agent,Tel
0,Flat,"Rotherhithe New Road, London SE16",3 bed flat to rent,3.0,1.0,1842.0,"Marketed by Merlin Cooper, SW1W",020 8128 1578
1,Studio,"Chingford Road, London E17",Studio to rent,1.0,1.0,750.0,"Marketed by Next Location Ltd Co Ltd, N16",020 8022 0049
2,Flat,"Gateway, Walworth Road, London SE17",5 bed flat to rent,5.0,1.0,2708.0,"Marketed by Merlin Cooper, SW1W",020 8128 1578
3,Flat,"Queens Drive, London N4",3 bed flat to rent,3.0,1.0,1770.0,"Marketed by Blackstones Estates, N8",020 3641 4305
4,Flat,"Milton Court, Chesterton Close, Wandsworth SW18",1 bed flat to rent,1.0,NaN,1500.0,"Marketed by Lets Find A Home, SW18",020 3641 0386


In [3]:
def geo_code(row):
    #from geopy.extra.rate_limiter import RateLimiter
    from geopandas.tools import geocode
    from dotenv import DotEnv

    key = DotEnv('key.env')
    
    try:
        point= geocode(row + ', United Kingdom', provider='mapquest', api_key=key.data['Key']).geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

cols = ['Latitude', 'Longitude', 'geometry']
data[cols] = data.apply(lambda x : geo_code(x['Address']), axis=1)
data.head()

,PropertyType,Address,Description,Bedrooms,Bathrooms,Price GBP,Agent,Tel,Latitude,Longitude,geometry
0,Flat,"Rotherhithe New Road, London SE16",3 bed flat to rent,3.0,1.0,1842.0,"Marketed by Merlin Cooper, SW1W",020 8128 1578,51.49164,-0.04712,POINT (-0.04712 51.49164)
1,Studio,"Chingford Road, London E17",Studio to rent,1.0,1.0,750.0,"Marketed by Next Location Ltd Co Ltd, N16",020 8022 0049,51.59095,-0.01768,POINT (-0.01768 51.59095)
2,Flat,"Gateway, Walworth Road, London SE17",5 bed flat to rent,5.0,1.0,2708.0,"Marketed by Merlin Cooper, SW1W",020 8128 1578,51.48497,-0.09377,POINT (-0.09377000000000001 51.48497)
3,Flat,"Queens Drive, London N4",3 bed flat to rent,3.0,1.0,1770.0,"Marketed by Blackstones Estates, N8",020 3641 4305,51.56248,-0.09558,POINT (-0.09558 51.56248)
4,Flat,"Milton Court, Chesterton Close, Wandsworth SW18",1 bed flat to rent,1.0,NaN,1500.0,"Marketed by Lets Find A Home, SW18",020 3641 0386,51.45735,-0.19938,POINT (-0.19938 51.45735)


In [6]:

success = (1 - data.Latitude.isna().sum() / len(data)) * 100
print(f'{int(success)}% of postcodes were goecoded!')

99% of postcodes were goecoded!


In [8]:
data.to_csv('rent_properties_London.csv', index=False)